In [ ]:
!git clone https://github.com/karoldvl/ESC-50

fatal: destination path 'ESC-50' already exists and is not an empty directory.


In [ ]:
import IPython.display as display

display.Audio('ESC-50/audio/1-100032-A-0.wav')

ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [ ]:
import glob
from collections import Counter

esc50_list = [f.split("-")[-1].replace(".wav", "") for f in glob.glob("ESC-50/audio/*.wav")]
print(esc50_list)
print(Counter(esc50_list))

In [ ]:
!cat /etc/os-release

In [ ]:
!apt install sox

In [ ]:
! pip install torchaudio

In [ ]:
import torchaudio.transforms as T
import torchaudio
from torch.utils.data import Dataset, DataLoader

class ESC50(Dataset):

    def __init__(self, path, transform=None):
        files = Path(path).glob('*.wav')
        self.items = [(f, int(f.name.split("-")[-1].replace(".wav", ""))) for f in files]
        self.length = len(self.items)
        self.transform = transform

    def __getitem__(self, index):
        filename, label = self.items[index]
        audio, sample_rate = torchaudio.load(filename)

        if self.transform:
            audio = self.transform(audio)

        return audio, label

    def __len__(self):
        return self.length

# Spectrogram transformation setup
spectrogram_transform = T.Spectrogram(
    n_fft=512,
    hop_length=256,
    power=2
)

In [ ]:
! mkdir /content/ESC-50/train /content/ESC-50/valid /content/ESC-50/test

In [ ]:
! mv /content/ESC-50/audio/1* /content/ESC-50/train
! mv /content/ESC-50/audio/2* /content/ESC-50/train
! mv /content/ESC-50/audio/3* /content/ESC-50/train
! mv /content/ESC-50/audio/4* /content/ESC-50/valid
! mv /content/ESC-50/audio/5* /content/ESC-50/test

In [ ]:
### get the data from the path and create a custom dataset for each
test_esc50 = "/content/ESC-50/test"
train_esc50 = "/content/ESC-50/train"
valid_esc50 = "/content/ESC-50/valid"

test_dataset = ESC50(path=test_esc50, transform=spectrogram_transform)
train_dataset = ESC50(path=train_esc50, transform=spectrogram_transform)
valid_dataset = ESC50(path=valid_esc50, transform=spectrogram_transform)

### create the data loader for each of them
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [ ]:
print(len(test_loader))
print(len(train_loader))
print(len(valid_loader))

In [ ]:
print(test_dataset[0])
audio, label = test_dataset[0]
print(audio.shape)
print(label)

In [ ]:
import matplotlib.pyplot as plt
import torchaudio.transforms as T

# Spectrogram transformation setup
spectrogram_transform = T.Spectrogram(
    n_fft=2048,
    hop_length=512,
    power=2  # Calculate the power spectrogram
)

# Function to plot waveform and spectrogram
def plot_waveform_and_spectrogram(audio, label):
    """
    Plots both the waveform and its corresponding spectrogram.

    Args:
        audio (torch.Tensor): Input audio tensor (waveform or spectrogram).
        label (str/int): Label associated with the audio.
    """
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    if audio.dim() == 2 and audio.size(0) == 1:  # Single-channel waveform [1, time]
        waveform = audio.squeeze(0).numpy()

        # Plot waveform
        axs[0].plot(waveform)
        axs[0].set_title(f"Waveform - Label: {label}")
        axs[0].set_xlabel("Time (samples)")
        axs[0].set_ylabel("Amplitude")

        # Generate spectrogram
        spectrogram = spectrogram_transform(audio)
    elif audio.dim() == 3 and audio.size(0) == 1:  # Single-channel spectrogram [1, freq, time]
        spectrogram = audio.squeeze(0)

        # Remove the waveform plot since the input is already a spectrogram
        fig.delaxes(axs[0])
        axs = [None, axs[1]]
    else:
        raise ValueError(f"Unexpected audio tensor shape: {audio.shape}")

    # Plot spectrogram
    im = axs[1].imshow(
        spectrogram.log2().detach().numpy(),
        cmap='viridis',
        aspect='auto',
        origin='lower'
    )
    fig.colorbar(im, ax=axs[1])
    axs[1].set_title(f"Spectrogram - Label: {label}")
    axs[1].set_xlabel("Time Bins")
    axs[1].set_ylabel("Frequency Bins")

    plt.tight_layout()
    plt.show()

# Loop through the first three samples in the test dataset
for i in range(3):
    audio, label = test_dataset[i]
    print(f"Processing Sample {i+1}: Audio Shape {audio.shape}, Label: {label}")
    plot_waveform_and_spectrogram(audio, label)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Halves the dimensions at each pooling layer

        # Dynamically calculate the input size for the fully connected layer
        self.fc1_input_size = self._get_conv_output_size((1, 257, 862))
        self.fc1 = nn.Linear(self.fc1_input_size, 500)
        self.fc2 = nn.Linear(500, 10)  # Adjust number of classes if needed

    def _get_conv_output_size(self, shape):
        """
        Pass a dummy tensor through conv layers to calculate the flattened output size.
        """
        dummy_input = torch.rand(1, *shape)  # Create a dummy tensor with input shape
        dummy_output = self.pool(F.relu(self.conv1(dummy_input)))
        dummy_output = self.pool(F.relu(self.conv2(dummy_output)))
        dummy_output = self.pool(F.relu(self.conv3(dummy_output)))
        return int(torch.numel(dummy_output))  # Total number of features after flattening

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # First convolution + pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second convolution + pooling
        x = self.pool(F.relu(self.conv3(x)))  # Third convolution + pooling
        x = x.view(x.size(0), -1)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Fully connected layer 2 (output)
        return x

In [ ]:
# Instantiate the model
model = AudioClassifier()

# Test with dummy input
dummy_input = torch.rand(32, 1, 257, 862)  # Batch size = 64
output = model(dummy_input)
print(f"Model output shape: {output.shape}")  # Should be (64, 10)

In [ ]:
from torch import optim
import torch

# Define the model, loss criterion, optimizer, and set hyperparameters
model = AudioClassifier()
criterion = nn.CrossEntropyLoss()
num_epochs = 10
best_lr = 0.01  # Adjust based on learning rate finder results
optimizer = optim.Adam(model.parameters(), lr=best_lr)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Compute model output
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model weights
        total_loss += loss.item()

    # Print average training loss per epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / len(train_loader)}')

    # Validation step
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for data, target in valid_loader:
            output = model(data)  # Compute model output
            loss = criterion(output, target)  # Calculate loss
            total_val_loss += loss.item()

    # Print average validation loss per epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {total_val_loss / len(valid_loader)}')